In [3]:
import numpy as np
from numpy import testing

In [5]:
%run numerics.ipynb

In [2]:
class MPDATA:
    def __init__(self, nr, r_min, r_max, dt, psi_r_lambda, coord):
        self._nm = numerics()
        
        assert r_min.units == r_max.units
        self.r_units = r_min.units
        self.t_units = dt.units
        
        self._n = 0

        #   |-----o-----|-----o--...
        # i-1/2   i   i+1/2   i+1
        # x_min     x_min+dx
        
        self._i = slice(1, nr+1)
        
        # cell-border stuff
        self._im = self._i % self._nm.hlf
        self._xm, self._dx = np.linspace(
            coord.x(r_min).magnitude, 
            coord.x(r_max).magnitude, 
            nr+1, 
            retstep=True
        )
        self._rm = coord.r(self._xm).to(self.r_units).magnitude
        self._Gm = coord.dx_dr(self._rm * self.r_units).magnitude
        self._GC = np.empty_like(self._Gm)
        
        # cell-centered stuff
        self._x = np.linspace(
            self._xm[0] - self._dx/2,
            self._xm[-1] + self._dx/2, 
            nr+2 
        )
        self._r = coord.r(self._x).to(self.r_units).magnitude
        
        self._G = np.empty(nr+2)
        self._G = coord.dx_dr(self._r * self.r_units).magnitude

        self._psi = np.empty_like(self._G)
        self._psi = (self._psi, self._psi.copy())
        self._psi[-1][self._i] = psi_r_lambda(self._r[self._i] * self.r_units).magnitude 
       
        # dt, dr
        self._dt = dt.magnitude
        self._dr = np.diff(self._r)
    
    @property
    def psi(self):
        return self._psi[self._n+1][self._i]

    @property
    def r(self):
        return self._r[self._i] * self.r_units
    
    def step(self, drdt_r_lambda):
        self._n = (self._n+2) % 2 - 1
        
        GC, G, Gm, psi, i, im, nm, n = self._GC, self._G, self._Gm, self._psi, self._i, self._im, self._nm, self._n
        
        # evaluating velocity
        GC[im] = Gm[im] * drdt_r_lambda(self._rm).to(self.r_units / self.t_units).magnitude / self._dt * self._dr
        testing.assert_array_less(np.amax(GC[im]), 1)
            
        # quick-n-dirty zero-gradient boundary condition
        # TODO: Jacobian!!!
        psi[n][i.start - 1] = psi[n][i.start]
        psi[n][i.stop] = psi[n][i.stop - 1]
        
        # upwind
        psi[n+1][i] = psi[n][i] - (
            nm.F(psi[n][i],        psi[n][i+nm.one], GC[i+nm.hlf]) - 
            nm.F(psi[n][i-nm.one], psi[n][i],        GC[i-nm.hlf])
        ) / G[i]